In [11]:
# Mount google drive to import and export data
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create base project folder
base_path = "/content/drive/MyDrive/Crisis_Intelligence_System"
folders = [
    "data/raw", "data/processed", "notebooks", "models",
    "outputs/visualizations", "outputs/metrics", "outputs/decisions",
    "dashboard/components", "utils"
]

for folder in folders:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)

print("Folder structure created at:", base_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder structure created at: /content/drive/MyDrive/Crisis_Intelligence_System


In [12]:
# Import necessary libraries
import pandas as pd
import geopandas as gpd

# Store raw data path to import files
RAW_DATA_PATH = "/content/drive/My Drive/Crisis_Intelligence_System/data/raw/"

# Load CSV files
disaster_events = pd.read_csv(os.path.join(RAW_DATA_PATH, "disaster_events.csv"))
economic_activity = pd.read_csv(os.path.join(RAW_DATA_PATH, "economic_activity.csv"))
energy_consumption = pd.read_csv(os.path.join(RAW_DATA_PATH, "energy_consumption.csv"))
events_calendar = pd.read_csv(os.path.join(RAW_DATA_PATH, "events_calendar.csv"))
local_reviews = pd.read_csv(os.path.join(RAW_DATA_PATH, "local_business_reviews.csv"))
sensor_readings = pd.read_csv(os.path.join(RAW_DATA_PATH, "sensor_readings.csv"))
social_media = pd.read_csv(os.path.join(RAW_DATA_PATH, "social_media_stream.csv"))
transportation = pd.read_csv(os.path.join(RAW_DATA_PATH, "transportation.csv"))
weather_data = pd.read_csv(os.path.join(RAW_DATA_PATH, "weather_historical.csv"))

# Load GeoJSON file
city_map = gpd.read_file(os.path.join(RAW_DATA_PATH, "city_map.geojson"))

**Sensor Data**

Step 1: Preprocess Sensor Data

In [13]:
# Load and filter sensor data
sensor_readings['timestamp'] = pd.to_datetime(sensor_readings['timestamp'])

# Keep only active sensors
sensor_readings = sensor_readings[sensor_readings['status'] == 'active'].copy()

# Reset index for cleanliness
sensor_readings.reset_index(drop=True, inplace=True)

# Assign zones based on latitude and longitude

# Get min/max of lat and lon
lat_min, lat_max = sensor_readings['latitude'].min(), sensor_readings['latitude'].max()
lon_min, lon_max = sensor_readings['longitude'].min(), sensor_readings['longitude'].max()

# Compute mid-points to split into 2x2 grid
lat_mid = (lat_min + lat_max) / 2
lon_mid = (lon_min + lon_max) / 2

# Function to assign zone
def assign_zone(row):
    if row['latitude'] >= lat_mid and row['longitude'] <= lon_mid:
        return "Zone A"
    elif row['latitude'] >= lat_mid and row['longitude'] > lon_mid:
        return "Zone B"
    elif row['latitude'] < lat_mid and row['longitude'] <= lon_mid:
        return "Zone C"
    else:
        return "Zone D"

# Apply to DataFrame
sensor_readings['zone'] = sensor_readings.apply(assign_zone, axis=1)

# Classify disaster based on sensor type
def map_disaster(sensor_type):
    mapping = {
        'flood': 'flood',
        'temperature': 'fire',
        'seismic': 'earthquake',
        'humidity': 'hurricane',
    }
    return mapping.get(sensor_type, None)

sensor_readings['disaster'] = sensor_readings['sensor_type'].apply(map_disaster)

# Function to classify severity into levels
def classify_disaster_level(score):
    if score < 50:
        return "Level 0: No Disaster"
    elif score < 65:
        return "Level 1: Mild Disaster"
    elif score < 80:
        return "Level 2: Moderate Disaster"
    elif score < 90:
        return "Level 3: Severe Disaster"
    else:
        return "Level 4: Critical Disaster"

# Apply to your sensor dataframe
sensor_readings['severity'] = sensor_readings['reading_value'].apply(classify_disaster_level)

# Drop unwanted columns
sensor_readings.drop(columns=['sensor_id', 'sensor_type', 'reading_value', 'status'], inplace=True, errors='ignore')

# Replace disaster with None where severity is Level 0
sensor_readings.loc[sensor_readings['severity'] == 'Level 0: No Disaster', 'disaster'] = None

# Preview data frame
sensor_readings.head()

,timestamp,latitude,longitude,zone,disaster,severity
0,2023-01-01 00:03:00,37.007146,-122.183781,Zone C,None,Level 0: No Disaster
1,2023-01-01 00:05:00,37.507472,-121.568884,Zone B,None,Level 0: No Disaster
2,2023-01-01 00:07:00,37.609489,-122.276630,Zone A,None,Level 0: No Disaster
3,2023-01-01 00:08:00,37.243154,-122.339465,Zone C,hurricane,Level 1: Mild Disaster
4,2023-01-01 00:15:00,37.963196,-122.376825,Zone A,None,Level 0: No Disaster


Step 2: Plot map of sensors

In [30]:
import pandas as pd
import folium

# Get min, max, and midpoints of lat/lon
lat_min, lat_max = sensor_readings['latitude'].min(), sensor_readings['latitude'].max()
lon_min, lon_max = sensor_readings['longitude'].min(), sensor_readings['longitude'].max()
lat_mid = (lat_min + lat_max) / 2
lon_mid = (lon_min + lon_max) / 2

# Define zone boundaries
zones = {
    "Zone A": [[lat_mid, lon_min], [lat_max, lon_mid]],
    "Zone B": [[lat_mid, lon_mid], [lat_max, lon_max]],
    "Zone C": [[lat_min, lon_min], [lat_mid, lon_mid]],
    "Zone D": [[lat_min, lon_mid], [lat_mid, lon_max]]
}

# Assign a color to each zone
zone_colors = {
    "Zone A": "red",
    "Zone B": "blue",
    "Zone C": "green",
    "Zone D": "purple"
}

# Create a folium map centered in the middle
map_center = [(lat_min + lat_max) / 2, (lon_min + lon_max) / 2]
m = folium.Map(location=map_center, zoom_start=11)

# Plot rectangles for each zone
for zone, ((lat1, lon1), (lat2, lon2)) in zones.items():
    bounds = [[lat1, lon1], [lat2, lon2]]
    folium.Rectangle(
        bounds=bounds,
        popup=zone,
        color=zone_colors[zone],
        fill=True,
        fill_color=zone_colors[zone],
        fill_opacity=0.3
    ).add_to(m)

m

Step 2: Preprocess Social Media Data

In [20]:
social_media['timestamp'] = pd.to_datetime(social_media['timestamp'])

# Use keyword matching to label tweets (you can use NLP later)
def classify_tweet(text):
    text = text.lower()
    if 'flood' in text:
        return 'flood'
    elif 'fire' in text:
        return 'fire'
    elif 'earthquake' in text:
        return 'earthquake'
    elif 'cyclone' in text or 'storm' or 'hurricane' in text:
        return 'cyclone'
    return None

social_media['disaster'] = social_media['text'].apply(classify_tweet)
social_media = social_media.dropna(subset=['disaster'])

Plot map for insfrastructure location from city map

In [42]:
import geopandas as gpd
import folium

# Define colors for each infrastructure type
color_map = {
    'hospital': 'blue',
    'fire_station': 'red',
    'shelter': 'green'
}

# Center map based on data
map_center = [city_map.geometry.centroid.y.mean(), city_map.geometry.centroid.x.mean()]
m = folium.Map(location=map_center, zoom_start=12)

# Plot pins for each type
for infra_type in city_map['type'].unique():
    subset = city_map[city_map['type'] == infra_type]
    for _, row in subset.iterrows():
        lat = row.geometry.centroid.y
        lon = row.geometry.centroid.x
        folium.Marker(
            location=[lat, lon],
            popup=infra_type.title(),
            icon=folium.Icon(color=color_map.get(infra_type, 'gray'), icon='info-sign')
        ).add_to(m)

m

<ipython-input-42-da38b3299299>:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_center = [city_map.geometry.centroid.y.mean(), city_map.geometry.centroid.x.mean()]


**Predict impact on infrastructure and recommmend action plan based on disaster**

In [14]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic

# Filter to the latest date only
latest_date = sensor_readings['timestamp'].dt.date.max()
sensor_filtered = sensor_readings[sensor_readings['timestamp'].dt.date == latest_date].copy()

# Drop rows with missing coordinates or severity
sensor_filtered = sensor_filtered.dropna(subset=['latitude', 'longitude', 'severity'])

# Rename geometry columns to avoid collision
city_map['infra_latitude'] = city_map.geometry.centroid.y
city_map['infra_longitude'] = city_map.geometry.centroid.x

# Rename for clarity
city_map = city_map.rename(columns={'type': 'infrastructure_type'})

# Cross join sensor readings and city map
sensor_filtered['key'] = 1
city_map['key'] = 1
cross_joined_df = pd.merge(sensor_filtered, city_map, on='key').drop('key', axis=1)

# Calculate geodesic distance
def calculate_distance(row):
    return geodesic(
        (row['latitude'], row['longitude']),
        (row['infra_latitude'], row['infra_longitude'])
    ).km

cross_joined_df['distance_km'] = cross_joined_df.apply(calculate_distance, axis=1)

# Predict impact level
def predict_impact(severity, distance_km):
    severity_level = int(severity.split(':')[0].split()[-1])  # e.g., "Level 3: Severe"

    if distance_km > 5:
        return 'Low'
    elif distance_km > 2:
        return 'Medium' if severity_level >= 2 else 'Low'
    else:
        if severity_level >= 4:
            return 'Critical'
        elif severity_level >= 3:
            return 'High'
        else:
            return 'Medium'

cross_joined_df['predicted_impact'] = cross_joined_df.apply(
    lambda row: predict_impact(row['severity'], row['distance_km']),
    axis=1
)

# Recommend action
def recommend_action(disaster, infra_type, impact, zone=None):
    if impact == "Critical":
        if infra_type == "shelter":
            return f"Evacuate {zone or 'this'} shelter immediately to alternate zone."
        elif infra_type == "hospital":
            return f"Reroute ambulances from hospital due to overload risk."
        elif infra_type == "fire_station":
            return f"Dispatch mobile units; station may be compromised."
    elif impact == "High":
        if infra_type == "hospital":
            return f"Scale up capacity or divert new cases from nearby areas."
        elif infra_type == "shelter":
            return f"Prepare secondary shelters and inform evacuees."
    elif impact == "Medium":
        return f"Monitor {infra_type} for escalation; notify emergency services."
    else:
        return f"No immediate action required."

cross_joined_df['recommended_action'] = cross_joined_df.apply(
    lambda row: recommend_action(row['disaster'], row['infrastructure_type'], row['predicted_impact'], row.get('zone')),
    axis=1
)

# Assign cross_joined_df to final_df
final_df = cross_joined_df  # This line is added

# Sort by timestamp
final_df = final_df.sort_values(by='timestamp').reset_index(drop=True)

# Drop geometry column as we have extracted latitude and longitude
final_df = final_df.drop(columns=['geometry'])

# Preview the result
final_df.head()

<ipython-input-14-5d58e97f4c85>:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  city_map['infra_latitude'] = city_map.geometry.centroid.y
<ipython-input-14-5d58e97f4c85>:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  city_map['infra_longitude'] = city_map.geometry.centroid.x


,timestamp,latitude,longitude,zone,disaster,severity,name,infrastructure_type,infra_latitude,infra_longitude,distance_km,predicted_impact,recommended_action
0,2023-02-04 00:01:00,37.86491,-121.593149,Zone B,None,Level 0: No Disaster,Location 0,hospital,37.432842,-122.492615,92.738142,Low,No immediate action required.
1,2023-02-04 00:01:00,37.86491,-121.593149,Zone B,None,Level 0: No Disaster,Location 658,shelter,37.871833,-122.187191,52.274854,Low,No immediate action required.
2,2023-02-04 00:01:00,37.86491,-121.593149,Zone B,None,Level 0: No Disaster,Location 659,fire_station,37.671539,-122.364318,71.255238,Low,No immediate action required.
3,2023-02-04 00:01:00,37.86491,-121.593149,Zone B,None,Level 0: No Disaster,Location 660,hospital,37.227079,-121.670372,71.119761,Low,No immediate action required.
4,2023-02-04 00:01:00,37.86491,-121.593149,Zone B,None,Level 0: No Disaster,Location 661,shelter,37.530766,-122.330168,74.834801,Low,No immediate action required.


In [126]:
import geopandas as gpd
import folium

# Define colors for each infrastructure type
color_map = {
    'hospital': 'blue',
    'fire_station': 'red',
    'shelter': 'green'
}

# Filter final_df for flood-related disasters with Level 1+ impact
relevant_impacts = ['Medium', 'High', 'Critical']
flood_df = final_df[(final_df['disaster'] == 'flood') & (final_df['predicted_impact'].isin(relevant_impacts))]

# Create map centered on affected area
map_center = [flood_df['infra_latitude'].mean(), flood_df['infra_longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=12)

# Plot markers for each affected infrastructure point
for _, row in flood_df.iterrows():
    infra_type = row['infrastructure_type']
    lat = row['infra_latitude']
    lon = row['infra_longitude']
    popup_text = f"{infra_type.title()}<br>Impact: {row['predicted_impact']}<br>Action: {row['recommended_action']}"

    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color=color_map.get(infra_type, 'gray'), icon='info-sign')
    ).add_to(m)

m

Save files in processed data for visualization

In [16]:
# Define the folder path
folder_path = "/content/drive/MyDrive/Crisis_Intelligence_System/data/processed/"

# Optional: Create the folder if it doesn't exist
import os
os.makedirs(folder_path, exist_ok=True)

# Save sensor_readings
sensor_filtered.to_csv(f"{folder_path}sensor_filtered.csv", index=False)

# Save final_df (impact + actions)
final_df.to_csv(f"{folder_path}final_df.csv", index=False)